In [ ]:
# 📊 YouTube動画収集ノートブック（国民民主党・参政党）
# 対象期間：2025年7月3日〜7月14日

# ✅ 必要なライブラリのインストール
!pip install --quiet google-api-python-client pandas

In [ ]:
API_KEY = ''

In [ ]:
# ✅ YouTube APIの初期設定と検索関数
from googleapiclient.discovery import build
import pandas as pd
import time

youtube = build('youtube', 'v3', developerKey=API_KEY)

def search_videos(query, published_after, published_before, max_results=2700):
    video_ids = []
    next_page_token = None
    while len(video_ids) < max_results:
        response = youtube.search().list(
            q=query,
            type='video',
            part='id',
            maxResults=50,
            publishedAfter=published_after,
            publishedBefore=published_before,
            relevanceLanguage='ja',
            regionCode='JP',
            pageToken=next_page_token
        ).execute()

        for item in response['items']:
            video_ids.append(item['id']['videoId'])

        next_page_token = response.get('nextPageToken')
        if not next_page_token:
            break
        time.sleep(0.1)

    return video_ids


In [ ]:
# ✅ 詳細データ取得関数
def get_video_details(video_ids):
    all_data = []
    for i in range(0, len(video_ids), 50):
        response = youtube.videos().list(
            part='snippet,statistics,contentDetails',
            id=','.join(video_ids[i:i+50])
        ).execute()

        for item in response['items']:
            data = {
                'videoId': item['id'],
                'title': item['snippet']['title'],
                'description': item['snippet']['description'],
                'channelTitle': item['snippet']['channelTitle'],
                'publishedAt': item['snippet']['publishedAt'],
                'viewCount': int(item['statistics'].get('viewCount', 0)),
                'duration': item['contentDetails']['duration']  # ISO 8601 format (e.g., 'PT5M30S')
            }
            all_data.append(data)
        time.sleep(0.1)
    return pd.DataFrame(all_data)


In [ ]:
# ✅ 検索対象キーワードリスト
kdp_keywords = ['国民民主党', '国民民主', '玉木']
ssp_keywords = ['参政党', '参政','神谷']

published_after = '2025-07-03T00:00:00Z'
published_before = '2025-07-19T23:59:59Z'

# ✅ 国民民主党関連の動画をすべて収集
kdp_ids = set()
for kw in kdp_keywords:
    ids = search_videos(kw, published_after, published_before, max_results=600)
    kdp_ids.update(ids)  # 重複を自動除去
kdp_ids = list(kdp_ids)
kdp_df = get_video_details(kdp_ids)
kdp_df.to_csv('kdp_youtube.csv', index=False, encoding='utf-8-sig')
print(f'国民民主党の動画数（重複排除後）: {len(kdp_df)} 件')


国民民主党の動画数（重複排除後）: 1029 件


In [ ]:
# ✅ 参政党関連の動画をすべて収集
ssp_ids = set()
for kw in ssp_keywords:
    ids = search_videos(kw, published_after, published_before, max_results=600)
    ssp_ids.update(ids)
ssp_ids = list(ssp_ids)
ssp_df = get_video_details(ssp_ids)
ssp_df.to_csv('ssp_youtube.csv', index=False, encoding='utf-8-sig')
print(f'参政党の動画数（重複排除後）: {len(ssp_df)} 件')


参政党の動画数（重複排除後）: 1009 件


In [ ]:
# 国民民主党の動画を表示（上位5件）
print("国民民主党の動画データ（例）:")
display(kdp_df.head())

国民民主党の動画データ（例）:


,videoId,title,description,channelTitle,publishedAt,viewCount,duration
0,AELcUB5ylPA,『news23』 小川彩佳アナが秋葉原で玉木雄一郎代表へ直撃姿！”自民圧力で降板”のウワサを...,動画生成: NoLang (no-lang.com)\nVOICEVOX: ずんだもん,話題のニュース,2025-07-16T12:28:51Z,18152,PT47S
1,BerBC5pqktE,【太田光が問う／国民民主党】玉木雄一郎代表ホンネ対談 選挙の日、そのまえに。｜TBS NEW...,7月20日投開票の参議院選挙を前に、爆笑問題・太田光が「選挙の日2025」のスペシャルキャス...,TBS NEWS DIG Powered by JNN,2025-07-14T07:40:00Z,294949,PT30M10S
2,AaPjakO0Ou8,榛葉幹事長最後の訴え#国民民主党 #玉木雄一郎 #榛葉賀津也 #奥村よしひろ＃牛田まゆ#sh...,2025年7月19日新橋SL広場街頭演説。\n\n引用元動画\nhttps://www.yo...,国民民主党私設応援団 凡事徹底,2025-07-19T13:08:19Z,33171,PT58S
3,4Y3w1FqT388,【衝撃】神谷代表、玉木さんにエグい協力要請してしまう,下記の出典に基づき解説しています\n■出典\nhttps://x.com/jinkamiya...,ガレ速報,2025-07-16T07:03:09Z,277962,PT3M32S
4,VER_tZhp6aU,#榛葉賀津也 国民もう気がついたよね！ 【国民民主党 参議院選挙2025 比例名簿政策】,国民に税金を返して経済を元気にする。消費税率下げてもみんなが元気になって買い物すれば、収入増...,平戸航太チャンネル【国民民主党 参議院 比例代表公認】,2025-07-18T06:00:26Z,36144,PT59S


In [ ]:
display(ssp_df.head())

,videoId,title,description,channelTitle,publishedAt,viewCount,duration
0,w2wk_SkDJvM,【参政党】「日本人ファースト」って何だ？過去に過激発言なぜ？他党と組む可能性は？神谷宗幣代表...,.\n◆ABEMAで無料視聴\n▷https://abema.go.link/gCY4N\n...,ABEMA Prime #アベプラ【公式】,2025-07-09T10:30:41Z,429047,PT35M47S
1,4Y3w1FqT388,【衝撃】神谷代表、玉木さんにエグい協力要請してしまう,下記の出典に基づき解説しています\n■出典\nhttps://x.com/jinkamiya...,ガレ速報,2025-07-16T07:03:09Z,277969,PT3M32S
2,FhoexTtJ_Uo,【ホリエモン】参政党大フィーバー神谷宗幣の謎,ご視聴いただきありがとうございます。\nチャンネル登録、高評価よろしくお願いいたします！\n...,サクッと ! ホリエモン【切り抜き】,2025-07-19T16:11:16Z,130668,PT35S
3,270Nios9i54,【医師が説明】参政党の医療政策のウソを分かりやすく解説！,参政党が話題になっています。神谷代表が語る医療関係の公約について説明します。終末期医療や看取...,病院グループ総帥 中田総院長,2025-07-19T03:57:22Z,26206,PT37M23S
4,-TT8h4_9GsI,【ひろゆき】参政党神谷代表の子供産めない問題発言について、正直に言います。【切り抜き】,#ひろゆき #ひろゆき切り抜き #論破 #解説 #ひろゆきの広場 #聞き流し #shorts...,ひろゆきの広場【ひろゆき切り抜き】,2025-07-07T08:00:00Z,9333,PT2M53S


In [ ]:
from google.colab import files

# 再保存（エンコーディング付き）
kdp_df.to_csv('kdp2_youtube.csv', index=False, encoding='utf-8-sig')

# ダウンロード
files.download('kdp_youtube.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files

# 再保存（エンコーディング付き）
ssp_df.to_csv('ssp2_youtube.csv', index=False, encoding='utf-8-sig')

# ダウンロード
files.download('ssp_youtube.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# 国民民主党の再生数・チャンネル数
print("▶ 国民民主党（kdp_df）")
print("総再生数:", kdp_df['viewCount'].sum())
print("平均再生数:", round(kdp_df['viewCount'].mean(), 2))
print("ユニークなチャンネル数:", kdp_df['channelTitle'].nunique())
print("チャンネル名一覧（上位10件）:")
print(kdp_df['channelTitle'].value_counts().head(10))


▶ 国民民主党（kdp_df）
総再生数: 91107122
平均再生数: 88539.48
ユニークなチャンネル数: 405
チャンネル名一覧（上位10件）:
channelTitle
国民民主党                          72
平戸航太チャンネル【国民民主党 参議院 比例代表公認】    45
国民民主党推し活ちゃんねる                  40
かわもと健一 国民民主党                   36
牛田まゆチャンネル                      26
日テレNEWS                        16
たまきチャンネル                       15
たまきカフェ                         14
奥村 よしひろ / Yoshihiro Okumura    14
ダークネスchan                      13
Name: count, dtype: int64


In [ ]:
# 国民民主党の再生数・チャンネル数
print("▶ 参政党（ssp_df）")
print("総再生数:", ssp_df['viewCount'].sum())
print("平均再生数:", round(ssp_df['viewCount'].mean(), 2))
print("ユニークなチャンネル数:", ssp_df['channelTitle'].nunique())
print("チャンネル名一覧（上位10件）:")
print(ssp_df['channelTitle'].value_counts().head(10))

▶ 参政党（ssp_df）
総再生数: 232197654
平均再生数: 230126.52
ユニークなチャンネル数: 463
チャンネル名一覧（上位10件）:
channelTitle
参政党                            46
喜べ!JAPAN                       21
ゴシップニュースジャパン                   18
チワワのウワサ【ネットの反応】                17
日テレNEWS                        14
ポメスキャンダル【ネットの反応】               11
橙色は希望の色                        11
【ニュース アメイジング】ガロン               10
悪の秘密ぼっち「ヘライザー総統」と言う名のファンタジー     9
れんけいのひとGAMES                    9
Name: count, dtype: int64
